In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
#export 
import numpy as np
from pathlib import Path
import pathlib
import os, yaml
from typing import  Union, Any
from fastprogress.fastprogress import MasterBar, ProgressBar
import pkg_resources
pkg_resources.require("fastprogress>=0.1.19")
from fastprogress.fastprogress import master_bar, progress_bar
import re
import requests
import tarfile


In [96]:
#export
PathOrStr = Union[Path,str]

def ifnone(a:Any,b:Any)->Any:
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a


In [97]:
#export
def download_url(url:str, dest:str, fname:str=None, overwrite:bool=False, pbar:ProgressBar=None,
                 show_progress=True, chunk_size=1024*1024, timeout=4, retries=5)->None:
    "Download `url` to `dest` unless it exists and not `overwrite`."
    if os.path.exists(dest) and not overwrite: return

    s = requests.Session()
    s.mount('http://',requests.adapters.HTTPAdapter(max_retries=retries))
    u = s.get(url, stream=True, timeout=timeout)
    try: 
        file_size = int(u.headers["Content-Length"])
    except: show_progress = False
    
    with open(dest, 'wb') as f:
        nbytes = 0
        if show_progress: pbar = progress_bar(range(file_size), auto_update=False, leave=False, parent=pbar)
        try:
            for chunk in u.iter_content(chunk_size=chunk_size):
                nbytes += len(chunk)
                if show_progress: pbar.update(nbytes)
                f.write(chunk)
        except requests.exceptions.ConnectionError as e:
            fname = url.split('/')[-1]
            from fastai.datasets import Config
            data_dir = Config().data_path()
            timeout_txt =(f'\n Download of {url} has failed after {retries} retries\n'
                          f' Fix the download manually:\n'
                          f'$ mkdir -p {data_dir}\n'
                          f'$ cd {data_dir}\n'
                          f'$ wget -c {url}\n'
                          f'$ tar -zxvf {fname}\n\n'
                          f'And re-run your code once the download is successful\n')
            print(timeout_txt)
            import sys;sys.exit(1)


In [101]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 02_fastai_imports.ipynb

Converted 02_fastai_imports.ipynb to exp/imports.py


In [106]:
download_url('https://data.4tu.nl/repository/uuid:3926db30-f712-4394-aebc-75976070e91f/DATA1','bpi18-2')

In [84]:
url='https://data.4tu.nl/repository/uuid:500573e6-accc-4b0c-9576-aa5468b10cee/DATA1'
s = requests.Session()
s.mount('http://',requests.adapters.HTTPAdapter(max_retries=5))
u = s.get(url, stream=True, timeout=4)

In [93]:
fname = ''
if "Content-Disposition" in u.headers.keys():
    fname = re.findall("filename=(.+)", u.headers["Content-Disposition"])[0]
else:
    fname = url.split("/")[-1]

print(fname)


"BPI_Challenge_2013_incidents.xes.gz"
